In [34]:
import pandas as pd

# ✅ 병합된 CSV 파일 경로
file_path = "../전처리_과정_데이터/병합완료.csv"
df = pd.read_csv(file_path, encoding='utf-8-sig')

print(df.head())

         발전구분  호기  설비용량(MW)  year  month  day  hour  발전량  발전효율
0  (군산)삼랑진태양광   1       2.0  2023      1    1     1  0.0   0.0
1  (군산)삼랑진태양광   1       2.0  2023      1    1     1  0.0   0.0
2  (군산)삼랑진태양광   1       2.0  2023      1    1     2  0.0   0.0
3  (군산)삼랑진태양광   1       2.0  2023      1    1     2  0.0   0.0
4  (군산)삼랑진태양광   1       2.0  2023      1    1     3  0.0   0.0


In [35]:
# ✅ 발전소 지점명 매핑 파일 불러오기
mapping_df = pd.read_csv('../전처리_과정_데이터/발전소지점명매핑.csv', encoding='utf-8-sig')

# ✅ 기상 데이터 파일 불러오기
weather_df = pd.read_csv('../전처리_과정_데이터/기상데이터통합.csv', encoding='utf-8-sig')

# ✅ 간단 확인
print("🗂️ 매핑 데이터 컬럼:", mapping_df.columns.tolist())
print("🗂️ 기상 데이터 컬럼:", weather_df.columns.tolist())


🗂️ 매핑 데이터 컬럼: ['발전소명', '지역명', '고도', '위도', '경도']
🗂️ 기상 데이터 컬럼: ['지점', '지점명', '일시', '기온(°C)', '기온 QC플래그', '강수량(mm)', '강수량 QC플래그', '습도(%)', '습도 QC플래그', '현지기압(hPa)', '현지기압 QC플래그', '일사(MJ/m2)', '일사 QC플래그', '적설(cm)', '전운량(10분위)', '중하층운량(10분위)', '운형(운형약어)', '최저운고(100m )', '풍속(m/s)', '풍속 QC플래그', '풍향(16방위)', '풍향 QC플래그']


C:\Users\박경국\AppData\Local\Temp\ipykernel_19116\3949919222.py:5: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  weather_df = pd.read_csv('../전처리_과정_데이터/기상데이터통합.csv', encoding='utf-8-sig')


In [37]:
import numpy as np

# ✅ 공백 제거
df.columns = df.columns.str.strip()
mapping_df.columns = mapping_df.columns.str.strip()

# ✅ 수동 매핑 적용
manual_mapping = {
    '신인천북측부지': '신인천 북측부지',
    '신인천주차장': '신인천 주차장',
    '이천시백사면A': '이천시 백사면A',
    '태안#9,10수상태양광': '태안#9,10 수상태양광',
}
df['발전구분'] = df['발전구분'].replace(manual_mapping)

# ✅ 불필요한 발전구분 제거
df = df[df['발전구분'] != '동해 바이오 연료저장고 지붕태양광']

# ✅ 병합 수행 (발전구분 ↔ 발전소명)
df = df.merge(mapping_df, left_on='발전구분', right_on='발전소명', how='left')

# ✅ 병합 결과 확인
missing_rows = df['지역명'].isna().sum()
print(f"❗ 병합 후 지역명 결측 행 수: {missing_rows}")
print("\n📋 병합된 결과 샘플:")
print(df[['발전구분', '지역명', '고도', '위도', '경도']].drop_duplicates().sort_values(by='발전구분').to_string(index=False))

❗ 병합 후 지역명 결측 행 수: 0

📋 병합된 결과 샘플:
         발전구분 지역명      고도        위도         경도
   (군산)삼랑진태양광 양산시   6.290 35.307370 129.020090
   광양항광양냉장태양광 광양시  88.210 34.943400 127.691400
    광양항어울림태양광 광양시  88.210 34.943400 127.691400
        남제주소내  제주  20.790 33.514110 126.529690
     농촌공진도태양광  목포  44.700 34.817320 126.381510
    당진자재창고태양광  서산  25.075 36.776557 126.493908
        당진태양광  서산  25.075 36.776557 126.493908
    당진화력수상태양광  서산  25.075 36.776557 126.493908
          무릉리  고산  71.390 33.293820 126.162830
     부산복합자재창고  부산  69.560 35.104680 129.032030
         부산본부  부산  69.560 35.104680 129.032030
       부산수처리장  부산  69.560 35.104680 129.032030
         부산신항  부산  69.560 35.104680 129.032030
        부산운동장  부산  69.560 35.104680 129.032030
         삼척소내 북강릉  75.240 37.804560 128.855350
 세종시폐기물매립장태양광  청주  59.390 36.639350 127.440890
          송당리  고산  71.390 33.293820 126.162830
   수원환경사업소태양광  수원  39.810 37.257460 126.983000
     신인천 북측부지  인천  68.990 37.477720 126.624900
      신인천 주차장  인천  68.990

In [44]:
from datetime import timedelta
import pandas as pd

# ✅ 1. 공백 제거 및 자료형 정리
df['지역명'] = df['지역명'].astype(str).str.strip()
weather_df['지점명'] = weather_df['지점명'].astype(str).str.strip()

# ✅ 2. 기상 데이터 '일시' 컬럼에서 datetime 분해
weather_df['일시'] = pd.to_datetime(weather_df['일시'], errors='coerce')
weather_df['year'] = weather_df['일시'].dt.year
weather_df['month'] = weather_df['일시'].dt.month
weather_df['day'] = weather_df['일시'].dt.day
weather_df['hour'] = weather_df['일시'].dt.hour

# ✅ 3. df의 hour 정수형 변환 및 24시 처리
df['hour'] = df['hour'].astype(int)
df['datetime'] = pd.to_datetime(df[['year', 'month', 'day']]) + pd.to_timedelta(df['hour'] - 1, unit='h')
df.loc[df['hour'] == 24, 'datetime'] += timedelta(hours=1)
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day
df['hour'] = df['datetime'].dt.hour
df.drop(columns='datetime', inplace=True)

# ✅ 4. 병합 수행 (지역명+시간 기준)
merged_df = df.merge(
    weather_df,
    left_on=['지역명', 'year', 'month', 'day', 'hour'],
    right_on=['지점명', 'year', 'month', 'day', 'hour'],
    how='left'
)

# 1. 병합 전에 df의 총 행 수 기억
original_len = len(df)

# 2. 병합된 행 수
merged_len = len(merged_df)

# 3. 병합 실패한 행 수 계산 (지점명+연월일+시간 기준으로 병합된 날씨 정보가 없었던 행들)
# 여기선 날씨 데이터에만 있는 고유 키 기준으로 병합 여부 판단
merged_success = merged_df[['지점명', 'year', 'month', 'day', 'hour']].notna().all(axis=1)
num_merged = merged_success.sum()
num_failed = original_len - num_merged

# 4. 결과 출력
print(f"🔎 원본 행 수: {original_len}")
print(f"✅ 병합된 행 수: {num_merged}")
print(f"❌ 병합 실패 행 수: {num_failed}")
print(f"🎯 병합 성공률: {100 * num_merged / original_len:.2f}%")




🔎 원본 행 수: 3206727
✅ 병합된 행 수: 3173169
❌ 병합 실패 행 수: 33558
🎯 병합 성공률: 98.95%


In [49]:
# ✅ 남기고 싶은 컬럼 목록
selected_cols = [
    '발전구분', '호기', '지점명', '위도', '경도', '설비용량(MW)', 'year', 'month', 'day', 'hour',
    '일사(MJ/m2)', '일사 QC플래그', '기온(°C)', '기온 QC플래그', '강수량(mm)', '강수량 QC플래그',
    '습도(%)', '습도 QC플래그', '적설(cm)',
    '전운량(10분위)', '풍속(m/s)', '풍속 QC플래그', '발전량', '발전효율'
]

# ✅ 컬럼 필터링
filtered_df = merged_df[selected_cols]

# ✅ 확인
print("✅ 필터링 후 컬럼 수:", filtered_df.shape[1])
print("✅ 컬럼 목록:", filtered_df.columns.tolist())


✅ 필터링 후 컬럼 수: 24
✅ 컬럼 목록: ['발전구분', '호기', '지점명', '위도', '경도', '설비용량(MW)', 'year', 'month', 'day', 'hour', '일사(MJ/m2)', '일사 QC플래그', '기온(°C)', '기온 QC플래그', '강수량(mm)', '강수량 QC플래그', '습도(%)', '습도 QC플래그', '적설(cm)', '전운량(10분위)', '풍속(m/s)', '풍속 QC플래그', '발전량', '발전효율']


In [50]:
filtered_df.head()

,발전구분,호기,지점명,위도,경도,설비용량(MW),year,month,day,hour,...,강수량(mm),강수량 QC플래그,습도(%),습도 QC플래그,적설(cm),전운량(10분위),풍속(m/s),풍속 QC플래그,발전량,발전효율
0,(군산)삼랑진태양광,1,양산시,35.30737,129.02009,2.0,2022,12,31,20,...,NaN,NaN,53.0,NaN,NaN,0.0,1.2,NaN,0.0,0.0
1,(군산)삼랑진태양광,1,양산시,35.30737,129.02009,2.0,2022,12,31,20,...,NaN,NaN,53.0,NaN,NaN,0.0,1.2,NaN,0.0,0.0
2,(군산)삼랑진태양광,1,양산시,35.30737,129.02009,2.0,2022,12,31,21,...,NaN,NaN,57.0,NaN,NaN,1.0,1.4,NaN,0.0,0.0
3,(군산)삼랑진태양광,1,양산시,35.30737,129.02009,2.0,2022,12,31,21,...,NaN,NaN,57.0,NaN,NaN,1.0,1.4,NaN,0.0,0.0
4,(군산)삼랑진태양광,1,양산시,35.30737,129.02009,2.0,2022,12,31,22,...,NaN,NaN,52.0,NaN,NaN,2.0,2.7,NaN,0.0,0.0


In [ ]:
# ✅ 저장 경로 설정
output_path = '../전처리_과정_데이터/기상추가완료.csv'

# ✅ CSV 파일로 저장
filtered_df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"✅ 파일이 다음 경로에 저장되었습니다: {output_path}")


✅ 파일이 다음 경로에 저장되었습니다: ../전처리_과정_데이터/기상추가완료.csv
